In [90]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import datetime
from sklearn.cluster import Birch
from sklearn.manifold import TSNE

In [91]:
df = pd.read_csv('data/janvier_2015.csv', sep=',',dtype={"latitude":float,"longitude":float,"wind_direction_true":float,"wind_speed":float,"present_weather":float,"sea_level_pressure":float,"air_temperature":float,"wetbulb_temperature":float,"sea_surface_temp":float},low_memory=False)

In [92]:
# delete index of all dataset
df = df.drop(df.columns[0],axis=1)
df_origin_data = df
df.head(3)

,latitude,longitude,wind_direction_true,wind_speed,present_weather,sea_level_pressure,air_temperature,wetbulb_temperature,sea_surface_temp,total_cloud_amount,cloud_height,wave_direction,wave_period,wave_height,swell_direction,swell_period,swell_height,timestamp
0,33.0,143.0,270.0,12.9,3.0,995.0,13.0,11.0,17.9,8.0,6,NaN,8.0,4.0,30.0,11.0,4.0,2005-01-31 17:45:00+00:00
1,43.4,-13.0,60.0,8.7,2.0,1029.9,12.6,NaN,13.9,6.0,6,NaN,15.0,4.0,13.0,15.0,4.0,2005-01-12 12:00:00+00:00
2,66.0,1.6,250.0,12.3,2.0,998.8,6.2,NaN,6.5,8.0,3,NaN,11.0,7.5,NaN,NaN,NaN,2005-01-29 20:00:00+00:00


In [93]:
def fill_NaN_and_String(df):
    df = df.replace('A',10)
    df.fillna(0,inplace=True)
    return df

In [94]:
def keep_only_temp(df):
    df = df.drop(["wind_direction_true","wind_speed","present_weather","sea_surface_temp","wetbulb_temperature","total_cloud_amount","cloud_height","wave_direction","wave_period","wave_height","swell_direction","swell_period","swell_height"],axis=1)
    return df

In [95]:
df = keep_only_temp(df)
df = df.dropna()
#df = fill_NaN_and_String(df)
df.head(10)

,latitude,longitude,sea_level_pressure,air_temperature,timestamp
0,33.0,143.0,995.0,13.0,2005-01-31 17:45:00+00:00
1,43.4,-13.0,1029.9,12.6,2005-01-12 12:00:00+00:00
2,66.0,1.6,998.8,6.2,2005-01-29 20:00:00+00:00
3,56.5,3.2,1013.2,7.1,2005-01-06 06:00:00+00:00
4,70.4,17.5,1010.2,4.0,2005-01-26 06:00:00+00:00
5,41.1,168.7,992.5,12.0,2005-01-31 23:45:00+00:00
6,41.3,-163.4,995.5,9.0,2005-01-30 12:00:00+00:00
7,61.1,1.0,988.8,5.7,2005-01-19 15:00:00+00:00
8,61.2,1.1,989.3,3.6,2005-01-02 13:00:00+00:00
9,61.2,1.1,981.5,3.9,2005-01-18 18:00:00+00:00


In [96]:
print(len(df))

334823


In [97]:
sea_level = []
air_temp = []
data = []
latitude = []
longitude = []

In [98]:
def keep_by_day(day):
    global sea_level,air_temp,data,latitude,longitude
    sea_level = []
    air_temp = []
    data = []
    latitude = []
    longitude = []
    for index, row in df.iterrows():
        storedTime = datetime.datetime.strptime(row["timestamp"][:-6], "%Y-%m-%d %H:%M:%S")
        if storedTime.day == day:
    #    if -8 < row['latitude'] < 62 and -86 < row['longitude'] < 12:
            sea_level.append(row['sea_level_pressure'])
            air_temp.append(row['air_temperature'])
            latitude.append(row['latitude'])
            longitude.append(row['longitude'])
            data.append(row)

In [99]:
# too long on all dataset
def fuse_balise(data):
    new_data = []
    while len(data) > 0:
        coord_to_erase = []
        print(len(data))
        storedTime = datetime.datetime.strptime(row["timestamp"][:-6], "%Y-%m-%d %H:%M:%S")
        new_row = []
        for index,coord2 in enumerate(data):
            if data[0]["latitude"] == coord2["latitude"] and data[0]["longitude"] == coord2["longitude"]:
                coord2["timestamp"] = storedTime.month
                coord2["cloud_height"] = float(coord2["cloud_height"])
                new_row.append(coord2)
                try:
                    data.pop(index)
                except IndexError:
                    print("sorry -- can't pop that")
        #print(coord_to_erase)
        new_row = np.array(new_row)
        #print(new_row)
        if len(new_row) > 1:
            new_data.append(np.mean(new_row))
        else:
            new_data.append(new_row)
    return new_data
                    

In [100]:
#temp = data
#machin = fuse_balise(temp)

In [101]:
def kmeans_on_pressure_temp():
    global sea_level,air_temp,data,latitude,longitude
    latitude = np.array(latitude)
    longitude = np.array(longitude)
    sea_level = np.array(sea_level)
    air_temp = np.array(air_temp)
    X = np.column_stack((sea_level, air_temp))

    #X = StandardScaler().fit_transform(dataZone)
    kmeans = KMeans(n_clusters=nb_clusters,random_state=1)
    kmeans.fit(X)
    
    idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
    lut = np.zeros_like(idx)
    lut[idx] = np.arange(nb_clusters)
    #lut[kmeans.labels_]
    y_kmeans = lut[kmeans.labels_] #kmeans.predict(X)
    
    return y_kmeans

#centers = kmeans.cluster_centers_
#plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

In [102]:
def show_graph(y_kmeans):
    pos = np.column_stack((latitude, longitude))
    plt.scatter(pos[:,0], pos[:, 1], c=y_kmeans, s=50, cmap='viridis')
    plt.show()

In [103]:
nb_clusters = 4

In [113]:
# m = folium.Map([0,0],zoom_start=1.5,prefer_canvas=True)
# m

In [ ]:
def tsne_algorithm():
    global sea_level,air_temp,data,latitude,longitude
    latitude = np.array(latitude)
    longitude = np.array(longitude)
    sea_level = np.array(sea_level)
    air_temp = np.array(air_temp)
    X = np.column_stack((latitude,longitude,sea_level, air_temp))
    X_embedded = TSNE(n_components=2,learning_rate=80,n_iter=300).fit_transform(X)
    plt.scatter(X_embedded[:,0], X_embedded[:, 1], cmap='viridis')
    plt.show()
#tsne_algorithm()

In [117]:
def plotDot(points, cluster):
    colors = ["red","blue","green","black"]
    for index,point in enumerate(points):
        for i in range(nb_clusters):
            if cluster[index] == i :
                folium.CircleMarker([point["latitude"], point["longitude"]],
                                radius=3,
                                weight=0.5, color=colors[i]).add_to(m)
            

In [119]:
nb_day = 31
for i in range(1,nb_day+1):
    title = "cluster_on_map_day{}.html".format(i)
    m = folium.Map([0, 0], zoom_start=1.5,prefer_canvas=True)
    keep_by_day(i)
    y_kmeans = kmeans_on_pressure_temp()
    plotDot(data,y_kmeans)
    m.save(title)


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
def show_scatter_graph(day):
    keep_by_day(day)
    y_kmeans = kmeans_on_pressure_temp()
    show_graph(y_kmeans)

In [ ]:
show_scatter_graph(10)

In [ ]:
#m = folium.Map([0, 0], zoom_start=1.5,prefer_canvas=True)
#keep_by_day(18)
#y_brc = birch_algorithm()
#plotDot(data,y_brc)
#m.save("test.html")